Questions:

What is distribution of volume spanned by single branch?
What is distribution of node spacing?
What is distribution of radii?
What is distribution of total nodes in a branch?
What is distribution of total nodes in a cell?

for each file

read line by line
extract coords
calc euclidean distance from prev point
keep track of min, max x/y/z
return span of x/y/z
calculate volume

In [3]:
import os

# get human trajectories
fnames = []
abs_paths = []
for root, dirs, fnames_ in os.walk("../data/human_splitted/"):
    fnames.extend(fnames_)
    for f in fnames_:
        relpath = os.path.join(root, f)
        abs_path = os.path.abspath(relpath)
        abs_paths.append(abs_path)


In [ ]:
import numpy as np
# https://stackoverflow.com/a/15934081/4212158
# catch np.sqrt RunTimeWarning as exception
np.seterr(all='raise')

dim_spans = []
node_distances = []
volumes = []
radii = []
n_nodes_per_branch = []
n_branches = 0
n_skipped = 0
fp_errors = 0

for fpath in abs_paths:  # for each branch
    min_x = np.inf
    min_y = np.inf
    min_z = np.inf
    max_x = -np.inf
    max_y = -np.inf
    max_z = -np.inf
    n_nodes = 0
    
    input = open(fpath, "r")
    prev = None  # the previous node
    for line in input:  # for each node
        n_nodes += 1
        
        # note: if the parent node is -1, then the child_node_id is the true identity of the root node
        try:
            _, _, x_coord, y_coord, z_coord, radius, _ = line.split()
        except ValueError:
            raise Exception("node is malformed")
        # convert string to f loat
        x_coord, y_coord, z_coord, radius = [np.float(x) for x in [x_coord, y_coord, z_coord, radius]]
        if x_coord < min_x:
            min_x = x_coord
        elif x_coord > max_x:
            max_x = x_coord
        if y_coord < min_x:
            min_y = y_coord
        elif y_coord > max_y:
            max_y = y_coord
        if z_coord < min_z:
            min_z = z_coord
        elif z_coord > max_z:
            max_z = z_coord
            
        if prev is not None:
            try:
                distance = np.sqrt((x_coord-prev["x"])**2 +
                              (y_coord-prev["y"])**2 +
                              (z_coord-prev["z"])**2)
            except FloatingPointError:
                #print(x_coord, prev["x"], y_coord, prev["y"], z_coord, prev["z"])
                fp_errors += 1
                print("x", x_coord-prev["x"]**2, type(x_coord))
                print("y", y_coord-prev["y"]**2)
                print("z", z_coord-prev["z"]**2, type(z_coord))
                print(distance)
            
            node_distances.append(distance)
        radii.append(radius)
            
        prev = {"x": x_coord, "y": y_coord, "z": z_coord}  # replace prev_node
        
    x_span = max_x - min_x
    y_span = max_y - min_y
    z_span = max_z - min_z
    volume = x_span * y_span * z_span
    
    volumes.append(volume)
    dim_spans.extend([x_span, y_span, z_span])
    n_nodes_per_branch.append(n_nodes)
    n_branches += 1
    
    #print("branch {} completed".format(n_branches))
            
    input.close()

branch 1 completed
branch 2 completed
branch 3 completed
branch 4 completed
branch 5 completed
branch 6 completed
branch 7 completed
branch 8 completed
branch 9 completed
branch 10 completed
branch 11 completed
branch 12 completed
branch 13 completed
branch 14 completed
branch 15 completed
branch 16 completed
branch 17 completed
branch 18 completed
branch 19 completed
branch 20 completed
branch 21 completed
branch 22 completed
branch 23 completed
branch 24 completed
branch 25 completed
branch 26 completed
branch 27 completed
branch 28 completed
branch 29 completed
branch 30 completed
branch 31 completed
branch 32 completed
branch 33 completed
branch 34 completed
branch 35 completed
branch 36 completed
branch 37 completed
branch 38 completed
branch 39 completed
branch 40 completed
branch 41 completed
branch 42 completed
branch 43 completed
branch 44 completed
branch 45 completed
branch 46 completed
branch 47 completed
branch 48 completed
branch 49 completed
branch 50 completed
branch 51

branch 693 completed
branch 694 completed
branch 695 completed
branch 696 completed
branch 697 completed
branch 698 completed
branch 699 completed
branch 700 completed
branch 701 completed
branch 702 completed
branch 703 completed
branch 704 completed
branch 705 completed
branch 706 completed
branch 707 completed
branch 708 completed
branch 709 completed
branch 710 completed
branch 711 completed
branch 712 completed
branch 713 completed
branch 714 completed
branch 715 completed
branch 716 completed
branch 717 completed
branch 718 completed
branch 719 completed
branch 720 completed
branch 721 completed
branch 722 completed
branch 723 completed
branch 724 completed
branch 725 completed
branch 726 completed
branch 727 completed
branch 728 completed
branch 729 completed
branch 730 completed
branch 731 completed
branch 732 completed
branch 733 completed
branch 734 completed
branch 735 completed
branch 736 completed
branch 737 completed
branch 738 completed
branch 739 completed
branch 740 co

branch 1293 completed
branch 1294 completed
branch 1295 completed
branch 1296 completed
branch 1297 completed
branch 1298 completed
branch 1299 completed
branch 1300 completed
branch 1301 completed
branch 1302 completed
branch 1303 completed
branch 1304 completed
branch 1305 completed
branch 1306 completed
branch 1307 completed
branch 1308 completed
branch 1309 completed
branch 1310 completed
branch 1311 completed
branch 1312 completed
branch 1313 completed
branch 1314 completed
branch 1315 completed
branch 1316 completed
branch 1317 completed
branch 1318 completed
branch 1319 completed
branch 1320 completed
branch 1321 completed
branch 1322 completed
branch 1323 completed
branch 1324 completed
branch 1325 completed
branch 1326 completed
branch 1327 completed
branch 1328 completed
branch 1329 completed
branch 1330 completed
branch 1331 completed
branch 1332 completed
branch 1333 completed
branch 1334 completed
branch 1335 completed
branch 1336 completed
branch 1337 completed
branch 133

branch 1923 completed
branch 1924 completed
branch 1925 completed
branch 1926 completed
branch 1927 completed
branch 1928 completed
branch 1929 completed
branch 1930 completed
branch 1931 completed
branch 1932 completed
branch 1933 completed
branch 1934 completed
branch 1935 completed
branch 1936 completed
branch 1937 completed
branch 1938 completed
branch 1939 completed
branch 1940 completed
branch 1941 completed
branch 1942 completed
branch 1943 completed
branch 1944 completed
branch 1945 completed
branch 1946 completed
branch 1947 completed
branch 1948 completed
branch 1949 completed
branch 1950 completed
branch 1951 completed
branch 1952 completed
branch 1953 completed
branch 1954 completed
branch 1955 completed
branch 1956 completed
branch 1957 completed
branch 1958 completed
branch 1959 completed
branch 1960 completed
branch 1961 completed
branch 1962 completed
branch 1963 completed
branch 1964 completed
branch 1965 completed
branch 1966 completed
branch 1967 completed
branch 196

branch 2442 completed
branch 2443 completed
branch 2444 completed
branch 2445 completed
branch 2446 completed
branch 2447 completed
branch 2448 completed
branch 2449 completed
branch 2450 completed
branch 2451 completed
branch 2452 completed
branch 2453 completed
branch 2454 completed
branch 2455 completed
branch 2456 completed
branch 2457 completed
branch 2458 completed
branch 2459 completed
branch 2460 completed
branch 2461 completed
branch 2462 completed
branch 2463 completed
branch 2464 completed
branch 2465 completed
branch 2466 completed
branch 2467 completed
branch 2468 completed
branch 2469 completed
branch 2470 completed
branch 2471 completed
branch 2472 completed
branch 2473 completed
branch 2474 completed
branch 2475 completed
branch 2476 completed
branch 2477 completed
branch 2478 completed
branch 2479 completed
branch 2480 completed
branch 2481 completed
branch 2482 completed
branch 2483 completed
branch 2484 completed
branch 2485 completed
branch 2486 completed
branch 248

...

branch 15551 completed
branch 15552 completed
branch 15553 completed
branch 15554 completed
branch 15555 completed
branch 15556 completed
branch 15557 completed
branch 15558 completed
branch 15559 completed
branch 15560 completed
branch 15561 completed
branch 15562 completed
branch 15563 completed
branch 15564 completed
branch 15565 completed
branch 15566 completed
branch 15567 completed
branch 15568 completed
branch 15569 completed
branch 15570 completed
branch 15571 completed
branch 15572 completed
branch 15573 completed
branch 15574 completed
branch 15575 completed
branch 15576 completed
branch 15577 completed
branch 15578 completed
branch 15579 completed
branch 15580 completed
branch 15581 completed
branch 15582 completed
branch 15583 completed
branch 15584 completed
branch 15585 completed
branch 15586 completed
branch 15587 completed
branch 15588 completed
branch 15589 completed
branch 15590 completed
branch 15591 completed
branch 15592 completed
branch 15593 completed
branch 1559

branch 16044 completed
branch 16045 completed
branch 16046 completed
branch 16047 completed
branch 16048 completed
branch 16049 completed
branch 16050 completed
branch 16051 completed
branch 16052 completed
branch 16053 completed
branch 16054 completed
branch 16055 completed
branch 16056 completed
branch 16057 completed
branch 16058 completed
branch 16059 completed
branch 16060 completed
branch 16061 completed
branch 16062 completed
branch 16063 completed
branch 16064 completed
branch 16065 completed
branch 16066 completed
branch 16067 completed
branch 16068 completed
branch 16069 completed
branch 16070 completed
branch 16071 completed
branch 16072 completed
branch 16073 completed
branch 16074 completed
branch 16075 completed
branch 16076 completed
branch 16077 completed
branch 16078 completed
branch 16079 completed
branch 16080 completed
branch 16081 completed
branch 16082 completed
branch 16083 completed
branch 16084 completed
branch 16085 completed
branch 16086 completed
branch 1608

branch 16569 completed
branch 16570 completed
branch 16571 completed
branch 16572 completed
branch 16573 completed
branch 16574 completed
branch 16575 completed
branch 16576 completed
branch 16577 completed
branch 16578 completed
branch 16579 completed
branch 16580 completed
branch 16581 completed
branch 16582 completed
branch 16583 completed
branch 16584 completed
branch 16585 completed
branch 16586 completed
branch 16587 completed
branch 16588 completed
branch 16589 completed
branch 16590 completed
branch 16591 completed
branch 16592 completed
branch 16593 completed
branch 16594 completed
branch 16595 completed
branch 16596 completed
branch 16597 completed
branch 16598 completed
branch 16599 completed
branch 16600 completed
branch 16601 completed
branch 16602 completed
branch 16603 completed
branch 16604 completed
branch 16605 completed
branch 16606 completed
branch 16607 completed
branch 16608 completed
branch 16609 completed
branch 16610 completed
branch 16611 completed
branch 1661

branch 17174 completed
branch 17175 completed
branch 17176 completed
branch 17177 completed
branch 17178 completed
branch 17179 completed
branch 17180 completed
branch 17181 completed
branch 17182 completed
branch 17183 completed
branch 17184 completed
branch 17185 completed
branch 17186 completed
branch 17187 completed
branch 17188 completed
branch 17189 completed
branch 17190 completed
branch 17191 completed
branch 17192 completed
branch 17193 completed
branch 17194 completed
branch 17195 completed
branch 17196 completed
branch 17197 completed
branch 17198 completed
branch 17199 completed
branch 17200 completed
branch 17201 completed
branch 17202 completed
branch 17203 completed
branch 17204 completed
branch 17205 completed
branch 17206 completed
branch 17207 completed
branch 17208 completed
branch 17209 completed
branch 17210 completed
branch 17211 completed
branch 17212 completed
branch 17213 completed
branch 17214 completed
branch 17215 completed
branch 17216 completed
branch 1721

branch 17948 completed
branch 17949 completed
branch 17950 completed
branch 17951 completed
branch 17952 completed
branch 17953 completed
branch 17954 completed
branch 17955 completed
branch 17956 completed
branch 17957 completed
branch 17958 completed
branch 17959 completed
branch 17960 completed
branch 17961 completed
branch 17962 completed
branch 17963 completed
branch 17964 completed
branch 17965 completed
branch 17966 completed
branch 17967 completed
branch 17968 completed
branch 17969 completed
branch 17970 completed
branch 17971 completed
branch 17972 completed
branch 17973 completed
branch 17974 completed
branch 17975 completed
branch 17976 completed
branch 17977 completed
branch 17978 completed
branch 17979 completed
branch 17980 completed
branch 17981 completed
branch 17982 completed
branch 17983 completed
branch 17984 completed
branch 17985 completed
branch 17986 completed
branch 17987 completed
branch 17988 completed
branch 17989 completed
branch 17990 completed
branch 1799

In [24]:
print(n_skipped)
print(fp_errors)

0
44347


In [19]:
import seaborn as sns

sns.distplot(dim_spans)

/home/shit/bin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shit/bin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


FloatingPointError: invalid value encountered in double_scalars